<a href="https://colab.research.google.com/github/AliTa-007/Datathon/blob/main/Coding%20is%20definitely%20an%20Art.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [51]:
# Importing all necessary libraries

import pandas as pd
import numpy as np
import datetime
import seaborn as sns
from sklearn.preprocessing import OneHotEncoder
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier

In [52]:
# Initializing the tables

xls = pd.ExcelFile('Data23.xlsx')
sales = pd.read_excel(xls, 'SALES')
products = pd.read_excel(xls, 'PRODUCTS')
customers = pd.read_excel(xls, 'CUSTOMERS')

In [53]:
# Merging all the tables into one

final =  sales.merge(products,on='ProductID').merge(customers,on='UserID')

In [54]:
# We will change some variables to a 0-1 form
# It is not necessary as Channel changes and we will build our model on the User dataframe

final["Gender"] = final.loc[:, "Gender"].replace({"FEMALE": 0, "MALE": 1})
final["Channel"] = final.loc[:, "Channel"].replace({"MOBILE": 0, "WEB": 1})

# final = pd.get_dummies(final, columns=['PaymentType'])

In [ ]:
# We call the transactions in the first 9-month period train and the rest is called test
# After we split the transactions table into the two desired time frames, we filter the "active" users

train = final.loc[final['DateTime'] < pd.Timestamp(2017,10,1), ]
train_active_users_transactions = train.loc[train['DateTime'] < pd.Timestamp(2017,7,1), ]
train_last_transaction_grpd = train[train["UserID"].isin(train_active_users_transactions["UserID"])].groupby("UserID")["DateTime"].agg(max)

test = final.loc[final['DateTime'] >= pd.Timestamp(2017,10,1), ]
test_active_users_transactions = test.loc[test['DateTime'] < pd.Timestamp(2018,4,1), ]
test_last_transaction_grpd = test[test["UserID"].isin(test_active_users_transactions["UserID"])].groupby("UserID")["DateTime"].agg(max)

# I will add the last transaction date to the users table for the desired members only
train_customers = customers.copy()
for i, date in enumerate(train_last_transaction_grpd):
  train_customers.loc[train_customers['UserID'] == train_last_transaction_grpd.index[i], "Last_Transaction_Date"] = date
train_customers = train_customers[~train_customers['Last_Transaction_Date'].isna()]

test_customers = customers.copy()
for i, date in enumerate(test_last_transaction_grpd):
  test_customers.loc[test_customers['UserID'] == test_last_transaction_grpd.index[i], "Last_Transaction_Date"] = date
test_customers = test_customers[~test_customers['Last_Transaction_Date'].isna()]

print(customers, train_customers, test_customers)

In [ ]:
# Change the train and test active users dfs' variables to numeric values

train_customers = pd.get_dummies(train_customers, columns=['Location'])
train_customers["Gender"] = train_customers.loc[:, "Gender"].replace({"FEMALE": 0, "MALE": 1})

test_customers = pd.get_dummies(test_customers, columns=['Location'])
test_customers["Gender"] = test_customers.loc[:, "Gender"].replace({"FEMALE": 0, "MALE": 1})

print(train_customers, test_customers)

In [ ]:
# We will create the churn variable for train and test users
train_customers["churn"] = np.where((train_customers["Last_Transaction_Date"] < pd.Timestamp(2017,7,1)), 1, 0)

test_customers["churn"] = np.where((test_customers["Last_Transaction_Date"] < pd.Timestamp(2018,4,1)), 1, 0)

print(train_customers, test_customers)

======================================================================
======================================================================


In [76]:

x_train = train_customers.drop('churn', axis = 1)

y_train = train_customers['churn']

x_test = test_customers.drop('churn', axis = 1)

y_test = test_customers['churn']



In [77]:
print(x_train, y_train, x_test, y_test)

         UserID    UserFirstTransaction  Gender  Age   Last_Transaction_Date  \
3     500234730 2017-01-24 00:00:00.000       1   33 2017-01-29 18:49:51.470   
4     500234752 2013-10-24 00:00:00.000       1   34 2017-09-05 18:10:24.850   
6     500234829 2009-09-01 00:00:00.000       1   42 2017-09-30 14:24:08.810   
7     500234840 2017-05-31 19:20:26.970       0   39 2017-05-31 19:22:57.230   
12    500235181 2010-03-25 00:00:00.000       0   40 2017-08-31 16:43:44.890   
...         ...                     ...     ...  ...                     ...   
3917  500611425 2004-05-20 00:00:00.000       0   42 2017-08-06 12:19:57.360   
3919  500611491 2015-10-29 00:00:00.000       1   27 2017-09-23 17:25:25.650   
3924  500612140 2016-09-19 00:00:00.000       1   24 2017-06-27 20:18:39.850   
3926  500612195 2011-02-26 00:00:00.000       1   46 2017-08-07 13:52:00.520   
3928  500612404 2017-06-09 18:57:14.750       0   46 2017-06-09 18:57:14.750   

      Location_ADANA  Location_ANKARA  